In [5]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [6]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.shape[0]

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


968557

In [12]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #276413
ppp_geonull.head()

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,1229 RICHARDSON HWY,DELTA JUNCTION,AK,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",1229 Richardson Hwy,DELTA JUNCTION,AK,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,1706 Ship Ave,Anchorage,AK,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,46877 STILLPOINT TRL,HALIBUT COVE,AK,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,1706 SHIP AVENUE,ANCHORAGE,AK,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN


In [13]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    32557
MI    26069
WA    23754
CO    19741
MN    19730
WI    17354
OR    13456
UT     9295
NE     5804
HI     4980
NM     4485
ID     4416
ND     3222
MT     3144
MO     3120
PR     2952
SD     2695
WY     2139
GU      443
RI      414
VI      290
MP       82
IN       28
NC       11
AL        8
PA        8
AK        5
AZ        4
NJ        3
MA        3
TN        3
FL        3
AR        2
ME        2
IL        2
IA        1
CT        1
MS        1
WV        1
GA        1
OH        1
Name: BorrowerState, dtype: int64

In [14]:
ppp_togeoWA = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "WA"]
ppp_togeoWA #23754

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
921945,921945,921945,1101807307,4/28/20,1013,PPP,COMPASS HEALTH,4526 FEDERAL AVE Building 4E,EVERETT,WA,...,999 Other,6214 Outpatient Care Centers,6214.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"4526 FEDERAL AVE Building 4E, EVERETT, WA",NaN,NaN,NaN
921946,921946,921946,1351807201,4/15/20,1013,PPP,NORTHWEST RESTAURANTS INC,18815 139th Ave NE Suite C,WOODINVILLE,WA,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"18815 139th Ave NE Suite C, WOODINVILLE, WA",NaN,NaN,NaN
921947,921947,921947,2545378910,4/27/21,1013,PPP,BLOODWORKS,921 Terry Ave,Seattle,WA,...,999 Other,6219 Other Ambulatory Health Care Services,6219.0,SEATTLE DISTRICT OFFICE,e $5-10 million,250 to 499,"921 Terry Ave, Seattle, WA",NaN,NaN,NaN
921948,921948,921948,2664207106,4/11/20,1013,PPP,"MOD SUPER FAST PIZZA, LLC",2035 158th Court NE 0.0,Bellevue,WA,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,SEATTLE DISTRICT OFFICE,e $5-10 million,500 or more,"2035 158th Court NE 0.0, Bellevue, WA",NaN,NaN,NaN
921949,921949,921949,3655697105,4/11/20,1013,PPP,"UNIVERSITY MECHANICAL CONTRACTORS, INC.",11611 49th Pl W,MUKILTEO,WA,...,999 Other,2382 Building Equipment Contractors,2382.0,SEATTLE DISTRICT OFFICE,e $5-10 million,250 to 499,"11611 49th Pl W, MUKILTEO, WA",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945694,945694,945694,9825088608,3/26/21,1013,PPS,SOUND OIL CO,6346 Rainier Ave S,Seattle,WA,...,999 Other,4543 Direct Selling Establishments,4543.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6346 Rainier Ave S, Seattle, WA",NaN,NaN,NaN
945695,945695,945695,9830848507,3/12/21,1013,PPS,AEROTECH COATINGS LLC,6021 199th St NE Ste A,Arlington,WA,...,336 Transportation Equipment,3364 Aerospace Product and Parts Manufacturing,3364.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6021 199th St NE Ste A, Arlington, WA",NaN,NaN,NaN
945696,945696,945696,9847407101,4/15/20,1013,PPP,BROWNING AND BROWNING INC,2145 hwy 101,PORT ANGELES,WA,...,999 Other,4842 Specialized Freight Trucking,4842.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2145 hwy 101, PORT ANGELES, WA",NaN,NaN,NaN
945697,945697,945697,9939018302,1/31/21,1013,PPS,CMP CONSTRUCTION,4720 158th Pl SW,Lynnwood,WA,...,999 Other,2361 Residential Building Construction,2361.0,SEATTLE DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4720 158th Pl SW, Lynnwood, WA",NaN,NaN,NaN


In [15]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoWA_add = ppp_togeoWA[cols]
ppp_togeoWA_add

,LoanNumber,full_add
921945,1101807307,"4526 FEDERAL AVE Building 4E, EVERETT, WA"
921946,1351807201,"18815 139th Ave NE Suite C, WOODINVILLE, WA"
921947,2545378910,"921 Terry Ave, Seattle, WA"
921948,2664207106,"2035 158th Court NE 0.0, Bellevue, WA"
921949,3655697105,"11611 49th Pl W, MUKILTEO, WA"
...,...,...
945694,9825088608,"6346 Rainier Ave S, Seattle, WA"
945695,9830848507,"6021 199th St NE Ste A, Arlington, WA"
945696,9847407101,"2145 hwy 101, PORT ANGELES, WA"
945697,9939018302,"4720 158th Pl SW, Lynnwood, WA"


In [16]:
add = ppp_togeoWA["full_add"].astype(str).to_list()

In [17]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{3}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('53')] #WA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/100.0.4896.60]


2035 158th Court NE 0.0, Bellevue, WA is Not Found
10502 NE 37th Cir Bldg 8, KIRKLAND, WA is Not Found
910 SW Spokane St, SEATTLE, WA is Not Found
PO Box 9970, SEATTLE, WA is Not Found
200 112 AVE NE; Suite 300, BELLEVUE, WA is Not Found
7440 MARGINAL WAY W, SEATTLE, WA is Not Found
4600 134TH PL S, TUKWILA, WA is Not Found
915 ANDERSON DR, ABERDEEN, WA is Not Found
1180 NW Maple Street 105, ISSAQUAH, WA is Not Found
1420 Fifth Ave, Suite 4200, SEATTLE, WA is Not Found
720 Olive Way Suite 1900, Seattle, WA is Not Found
130145 S Fawcett St, TACOMA, WA is Not Found
11621 East Marginal Way S, Suite A, Seattle, WA is Not Found
1505 NORTH MILLER ST SUITE 260, WENATCHEE, WA is Not Found
1301 FIFTH AVENUE SUITE 2700, SEATTLE, WA is Not Found
17425 NE Union Hill Rd., STE 250, REDMOND, WA is Not Found
17735 NE 65TH STREET SUITE 130, REMOND, WA is Not Found
14980 NE 31ST WAY STE 100, REDMOND, WA is Not Found
10510 NORTHUP WAY STE 350, KIRKLAND, WA is Not Found
134TH ST728 134th St SW, Ste 103, E

1221 Harbour Reach Dr. #200, MUKILTEO, WA is Not Found
3075 112th Ave NE Ste 200, BELLEVUE, WA is Not Found
2401 E 6th Street, DEER PARK, WA is Not Found
2300 Ron Crockett Dr NW, AUBURN, WA is Not Found
3201 S 176th St, Seatac, WA is Not Found
319 D St NW Ste 104, Auburn, WA is Not Found
7834 SE 32nd St Ste 101, Mercer Island, WA is Not Found
1180 NW Maple St Ste 105, Issaquah, WA is Not Found
2220 Lind Ave SW Ste 106 Integrity Networks Inc, Renton, WA is Not Found
3191 Wesley Road, White Swan, WA is Not Found
10200 Quil Ceda Blvd, Quil Ceda Village, WA is Not Found
1305 Tacoma Ave S Ste 201, Tacoma, WA is Not Found
7100 Fort Dent Way Ste 280, Tukwila, WA is Not Found
18420 E Garland Ave, Spokane Valley, WA is Not Found
2211 SE Grace Ave, Battle Ground, WA is Not Found
509 Olive Way Ste 305, Seattle, WA is Not Found
1936 Harbor Ave SW, Seattle, WA is Not Found
10502 NE 37th Cir Bldg 8, Kirkland, WA is Not Found
16700 NE 79th St Ste 100, Redmond, WA is Not Found
1001 Yakima Ave Ste 2, T

11715 N CREEK PKWY Suite C160, BOTHELL, WA is Not Found
319 D St NW Ste 104, Auburn, WA is Not Found
1000 2nd Avenue #1200, SEATTLE, WA is Not Found
2853 SE 27TH ST, MERCER ISLAND, WA is Not Found
1-100th Ave NE Suite 102, BELLEVUE, WA is Not Found
13810 SE Eastgate Way Ste 110, BELLEVUE, WA is Not Found
1900 NE 162nd Ave Ste D103, Vancouver, WA is Not Found
11111 NE 8th St Ste 125, Bellevue, WA is Not Found
3304 Old Harford Rd, LAKE STEVENS, WA is Not Found
11807 Northcreek Parkway South Suite B, BOTHELL, WA is Not Found
4424 98th St. Ct. SW Bdg 26 Suite B, LAKEWOOD, WA is Not Found
1900 NE 78th Street, Suite 101, VANCOUVER, WA is Not Found
270 S HANFORD ST 100, SEATTLE, WA is Not Found
900 16TH ST SW, RENTON, WA is Not Found
3211 Martin Luther King Jr Way S Suite B, SEATTLE, WA is Not Found
1730 MINOR AVE STE 1400, SEATTLE, WA is Not Found
1120 SW 16TH ST STE 4A, RENTON, WA is Not Found
2692 NE 49th St, SEATTLE, WA is Not Found
10013 NE Hazel Dell Ave, Suite 421, VANCOUVER, WA is Not

910 VALLEY AVE NW Unit 105, PUYALLUP, WA is Not Found
1813 30th Ave SW, Puyallup, WA is Not Found
PO Box 2789, BELLINGHAM, WA is Not Found
3610 ALBION PL N # 102, SEATTLE, WA is Not Found
810-3rd Ave Suite 500, Seattle, WA is Not Found
13810 SE Eastgate Way Ste 110, Bellevue, WA is Not Found
1745 NW Mall St, ISSAQUAH, WA is Not Found
4601 NE 77TH AVE STE 300, VANCOUVER, WA is Not Found
PO BOX 6470, SPOKANE, WA is Not Found
201 FRONT STREET, PACIFIC BEACH, WA is Not Found
11111 NE 8TH ST #125, BELLEVUE, WA is Not Found
14001 SE 1st St, VANCOUVER, WA is Not Found
14001 SE 1st St, Vancouver, WA is Not Found
16778 146TH ST SE STE 134, MONROE, WA is Not Found
12842 State Route 9, CLEARLAKE, WA is Not Found
12842 Highway 9, CLEAR LAKE, WA is Not Found
2021 NW Myhre Pl Ste 300, Silverdale, WA is Not Found
415 SE 117th Ave #108, VANCOUVER, WA is Not Found
415 SE 117th Ave Ste 108, Vancouver, WA is Not Found
12910 TOTEM LAKE BLVD NE, SUITE 200, KIRKLAND, WA is Not Found
12303 NE 56th Street, Br

...

15545-12th Ave NE, SEATTLE, WA is Not Found
13 West Moon Street, OTHELLO, WA is Not Found
16 Hikanson Rd, ELMA, WA is Not Found
300 COLLEENS WAY, GRANDVIEW, WA is Not Found
3511 135th St NE Ste 102, Marysville, WA is Not Found
10020 NE 72ND AVE SUITE 104, VANCOUVER, WA is Not Found
16209 SE McGillivray Blvd Ste T, Vancouver, WA is Not Found
8960 STATE ROUTE 2 STE 146, Monroe, WA is Not Found
15808 Mill Creek Blvd Ste 110, Mill Creek, WA is Not Found
16315 SMOKEY BLVD, MARYSVILLE, WA is Not Found
400 nw gilman blvd 1370, ISSAQUAH, WA is Not Found
P.O. Box 1007, everett, WA is Not Found
4510 NE 68th Dr, VANCOUVER, WA is Not Found
100 Ravine Ln NE Ste 310, Bainbridge Island, WA is Not Found
15080 Burley Ave SE, GIG HARBOR, WA is Not Found
14508 NE 20th St Ste 100, Bellevue, WA is Not Found
1301 S. Fair Ave, YAKIMA, WA is Not Found
2119 SE 192nd Ave Ste 100, Camas, WA is Not Found
801 NW 42ND ST STE 202, SEATTLE, WA is Not Found
2912 E Palouse Hwy Ste A, Spokane, WA is Not Found
7683 SE 27

2050 112th Ave Ne Suite 115, BELLEVUE, WA is Not Found
901 Boren Ave Ste 1500, Seattle, WA is Not Found
5908 BEDFORD ST STE B, PASCO, WA is Not Found
1037 NE 65Th Street 360, Seattle, WA is Not Found
1037 NE 65th St # 360, Seattle, WA is Not Found
41115 CHILTON DR NW, WOODLAND, WA is Not Found
PO Box 627, RAINIER, WA is Not Found
1517 S Fawcett 101B, TACOMA, WA is Not Found
2415 SE 165th Ave, Suite 105, VANCOUVER, WA is Not Found
2540 CASCADES PASS BLVD, BREMERTON, WA is Not Found
16500 SE 15th St Ste 180, Vancouver, WA is Not Found
110 Cedar Ave Apt 102, Snohomish, WA is Not Found
2670 1ST ST, RICHLAND, WA is Not Found
22820 NE 8th ST STE 102, Sammamish, WA is Not Found
6947 Coal Creek Pkwy SE PMB 186, Newcastle, WA is Not Found
PO Box 6241, EDMONDS, WA is Not Found
1420 NW Gilman Blvd Ste 2 # 8010, Issaquah, WA is Not Found
1513 18TH ST NW STE A, PUYALLUP, WA is Not Found
701 The Parkway Ste A, Richland, WA is Not Found
10735 A street S D, TACOMA, WA is Not Found
138 2661 N PEARL ST,

8151 NE 164th ave NE 422, Redmond, WA is Not Found
40 KINGS CT, CLE ELUM, WA is Not Found
21234 MIDDLE FORK RD, CHEHALIS, WA is Not Found
LAMBS EAR PL, PORT TOWNSEND, WA is Not Found
989 112TH AVE #105, BELLEVUE, WA is Not Found
1920 SE 167th Ave, Vancouver, WA is Not Found
98107, SEATTLE, WA is Not Found
17615 SE Cougar Mtn DR Ste. A, ISSAQUAH, WA is Not Found
1920 SE 167TH AVE, VANCOUVER, WA is Not Found
1205 CRAFTSMAN WAY STE 111, EVERETT, WA is Not Found
3525 Ensign Rd NE Ste J, Olympia, WA is Not Found
7302.5 15TH AVE NW, SEATTLE, WA is Not Found
2101 Caton Way SW Ste. 102, OLYMPIA, WA is Not Found
PO Box 112, MEDINA, WA is Not Found
4437 Russell Road Ste. 200, KENT, WA is Not Found
4750 DIVISION ST N, SPOKANE, WA is Not Found
16911 103rd Ave E, Puyallup, WA is Not Found
422 W Riverside, Suite 920, SPOKANE, WA is Not Found
250 Chardonnay Ave, Prosser, WA is Not Found
1525 A ST NE STE 100, AUBURN, WA is Not Found
940 Auburn Ct, SNOHOMISH, WA is Not Found
203 Connell Hill Rd, POMERO

18960 STATE ROUTE 2 Suite 146, MONROE, WA is Not Found
735 NW Gilman Blvd Ste E, Issaquah, WA is Not Found
15320 BROWN AVE NE, DUVALL, WA is Not Found
12501 Bel Red Road  #100, Bellevue, WA is Not Found
116 SW 148TH ST STE D100, BURIEN, WA is Not Found
3610 Cedardale Road Unit C, MOUNT VERNON, WA is Not Found
700 AVENUE D SUITE 102, SNOHOMISH, WA is Not Found
31611 PETE VON REICHBAUER WAY S., FEDERAL WAY, WA is Not Found
3300 CHESTNUT ST, EVERETT, WA is Not Found
14201 SE PETROVITSKY RD A3, RENTON, WA is Not Found
280 HARDIE AVE SW STE 3, RENTON, WA is Not Found
280 Hardie Ave SW Ste 3, Renton, WA is Not Found
701 Fifth Avenue, Suite 4640, Seattle, WA is Not Found
14711 NE 29th Pl #101, BELLEVUE, WA is Not Found
2620 N Hendrickson Drive, KALAMA, WA is Not Found
468 Argyle Ave ste B, FRIDAY HARBOR, WA is Not Found
1011 SW Klickitat Way Suite 201, SEATTLE, WA is Not Found
1037 NE 65th St PMB 80243, Seattle, WA is Not Found
927 40th Ave SW Ste B, Puyallup, WA is Not Found
7200 W Nob Hill 

6134 NE Bothell Way Ste C, Kenmore, WA is Not Found
14725 NE 20th St Ste D44, Bellevue, WA is Not Found
509 Olive Way Ste 1149, Seattle, WA is Not Found
1170 Sunset Blvd NE, Renton, WA is Not Found
4300 TALBOT RD S STE 300, RENTON, WA is Not Found
3228 LakeLeo Way, Langley, WA is Not Found
155 NE 100th Street, Suite 500, Seattle, WA is Not Found
19205 NE Union Hill Rd, Redmond, WA is Not Found
10735 A St S Ste D, Tacoma, WA is Not Found
2920 Colby Ave Ste 101, Everett, WA is Not Found
15731 NE 8th St Unit 7058, Bellevue, WA is Not Found
1621 Broadway Ave, SNOHOMISH, WA is Not Found
16911 103rd Avenue E Unit 102, PUYALLUP, WA is Not Found
4211 Aldewrwood Mall Blvd, Suite 206, LYNNWOOD, WA is Not Found
20412 258TH PL SE, COVINGTON, WA is Not Found
7721 Center Blvd SE 7721 Center Blvd Se, Snoqualmie, WA is Not Found
12516 NE 95th St. #D110, VANCOUVER, WA is Not Found
1037 NE 65th St # 196, Seattle, WA is Not Found
14531 ALTERNATE STATE ROUTE 97, ENTIAT, WA is Not Found
5224 Wilson Ave S. 

In [18]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

23754
23762
23754
23754


<b>Add remaining values (2801) to result_df(5461)

In [ ]:
lat_df = pd.DataFrame({"Lat": lat})
lat_df_sort = lat_df.sort_values(by="Lat")
lat_df_sort

In [ ]:
pd.set_option('display.max_rows', None)
add_df = pd.DataFrame({"add": add, "Lat": lat})
add_df

In [ ]:
result_df = pd.DataFrame({"Long":long, "FIPS_z": geo_id})
result_df

In [ ]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

In [ ]:
#mapping coordinates to NaN values in main 
m = dict(zip(result_df.full_add,result_df.Lat))
ppp_togeoMI.Lat = ppp_togeoMI.Lat.fillna(ppp_togeoMI.full_add.map(m))
n = dict(zip(result_df.full_add,result_df.Long))
ppp_togeoMI.Long = ppp_togeoMI.Long.fillna(ppp_togeoMI.full_add.map(n))
o = dict(zip(result_df.full_add,result_df.FIPS_z))
ppp_togeoMI.FIPS_z = ppp_togeoMI.FIPS_z.fillna(ppp_togeoMI.full_add.map(o))

In [ ]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoMI_parsed = ppp_togeoMI[cols]
ppp_togeoMI_parsed.head()

In [ ]:
ppp_geo = ppp_geo.merge(ppp_togeoMI_parsed, on="LoanNumber", how="left")
ppp_geo.head()

In [ ]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

In [ ]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")

In [ ]:
ppp_geo_mi = ppp_geo.loc[ppp_geo["BorrowerState"] =="MI"]
ppp_geo_mi.shape[0]

In [ ]:
null_check = ppp_geo_mi.loc[ppp_geo_mi["FIPS_z"].isnull()]
null_check.shape[0]